### Since we want sentiment analys on Customers only. I am taking approach
##### 1. If I extract customer only text then I loose lots of context. So, I am using sentiment analysis on full text and sentiment analysis on customer only text. 
##### 2. I am spliing the text into start, middle and last and then calcualting setiment on each segment. This way we can change the tresholds and overweigh/underwight to suit our purpose.


### Splitting each documents into 3 sections

In [97]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Function to split text into segments (e.g., start, middle, end)
def split_transcript(text, num_segments=3):
    # Split text into approximately equal segments
    sentences = text.split('. ')
    segment_size = len(sentences) // num_segments
    segments = [". ".join(sentences[i:i + segment_size]) for i in range(0, len(sentences), segment_size)]
    return segments[:num_segments]  # Return only the specified number of segments

# Function to get sentiment for each segment
def sentiment_per_segment(text):
    # Split text into 3 parts: start, middle, end
    segments = split_transcript(text, num_segments=3)
    
    segment_sentiments = []
    for segment in segments:
        # Tokenize the input text for each segment
        inputs = tokenizer(segment, return_tensors="pt", padding=True, truncation=True)

        # Get the logits from the model
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Convert logits to probabilities using softmax
        probs = F.softmax(logits, dim=-1).squeeze().tolist()

        # Combine label 1 and 2 as Negative, label 3 as Neutral, label 4 and 5 as Positive
        negative_prob = probs[0] + probs[1]  # Combine 1-star and 2-star for Negative
        neutral_prob = probs[2]               # 3-star is Neutral
        positive_prob = probs[3] + probs[4]   # Combine 4-star and 5-star for Positive

        # Append the sentiment for this segment
        segment_sentiments.append({
            "negative": negative_prob,
            "neutral": neutral_prob,
            "positive": positive_prob
        })

    return segment_sentiments

# Function to aggregate and interpret sentiment across segments
def aggregate_sentiment(sentiments):
    # Initialize total probabilities
    total_negative = 0
    total_neutral = 0
    total_positive = 0

    # Define a weight for the third sentiment
    third_section_weight = 1.5 

    # Sum up probabilities across segments
    for idx, sentiment in enumerate(sentiments):
        if idx == 2:  # Overweight the third sentiment
            total_negative += sentiment["negative"] * third_section_weight
            total_neutral += sentiment["neutral"] * third_section_weight
            total_positive += sentiment["positive"] * third_section_weight
        else:  # Regular sum for the other sentiments
            total_negative += sentiment["negative"]
            total_neutral += sentiment["neutral"]
            total_positive += sentiment["positive"]

    # Average the probabilities over the number of segments
    num_segments = len(sentiments)
    avg_negative = total_negative / num_segments
    avg_neutral = total_neutral / num_segments
    avg_positive = total_positive / num_segments

    
    sentiment_scores = {
    "negative": avg_negative,
    "neutral": avg_neutral,
    "positive": avg_positive
     }
    sentiment_label = max(sentiment_scores, key=sentiment_scores.get)

    return {
        "average_negative": avg_negative,
        "average_neutral": avg_neutral,
        "average_positive": avg_positive
        ,"predicted_sentiment": sentiment_label
    }


/Users/abhi/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Customer only Transcripts

In [98]:
import os

# Directory where customer-only transcripts are stored
customer_text_dir = './customer_only_transcripts/'

# Load all customer texts from the directory
customer_texts = []

for i in range(200):  
    file_path = os.path.join(customer_text_dir, f'transcript_{i}_customer.txt')
    with open(file_path, 'r', encoding='utf-8') as file:
        customer_texts.append(file.read())

In [99]:
customer_sentiments = []

for idx, text in enumerate(customer_texts, start=0):  # Start index from 0
    sentiment = sentiment_per_segment(text) 
    final_sentiment = aggregate_sentiment(sentiment)
    sentiment_entry = {"ID": idx, "Sentiment": final_sentiment}
    customer_sentiments.append(sentiment_entry)   


# Print the results
#for entry in customer_sentiments:
#    print(f"ID: {entry['ID']}, Sentiment: {entry['Sentiment']}")

In [100]:
customer_sentiments

[{'ID': 0,
  'Sentiment': {'average_negative': 0.30177441394577426,
   'average_neutral': 0.24598586683471999,
   'average_positive': 0.6189063688119253,
   'predicted_sentiment': 'positive'}},
 {'ID': 1,
  'Sentiment': {'average_negative': 0.6111348178237677,
   'average_neutral': 0.1653121088941892,
   'average_positive': 0.3902196964869897,
   'predicted_sentiment': 'negative'}},
 {'ID': 2,
  'Sentiment': {'average_negative': 0.3217562170078357,
   'average_neutral': 0.3874304989973704,
   'average_positive': 0.45747992520531017,
   'predicted_sentiment': 'positive'}},
 {'ID': 3,
  'Sentiment': {'average_negative': 0.9615673099954923,
   'average_neutral': 0.1142205260694027,
   'average_positive': 0.09087881884382416,
   'predicted_sentiment': 'negative'}},
 {'ID': 4,
  'Sentiment': {'average_negative': 0.3134390066067378,
   'average_neutral': 0.389510914683342,
   'average_positive': 0.46371672737101716,
   'predicted_sentiment': 'positive'}},
 {'ID': 5,
  'Sentiment': {'average_

### Full transcripts

In [101]:

full_text_dir = './transcripts_v3/'  # Directory where full transcripts are stored

# Load all customer texts from the directory
full_texts = []

for i in range(200):  
    file_path = os.path.join(full_text_dir, f'transcript_{i}.txt')
    with open(file_path, 'r', encoding='utf-8') as file:
        full_texts.append(file.read())

In [102]:
full_text_sentiments = []

for idx, text in enumerate(full_texts, start=0):  # Start index from 0
    sentiment = sentiment_per_segment(text)
    final_sentiment = aggregate_sentiment(sentiment)
    sentiment_entry = {"ID": idx, "Sentiment": final_sentiment}
    full_text_sentiments.append(sentiment_entry)


# Print the results
#for entry in full_text_sentiments:
    #print(f"ID: {entry['ID']}, Sentiment: {entry['Sentiment']}")


In [103]:
full_text_sentiments

[{'ID': 0,
  'Sentiment': {'average_negative': 0.5431260541081429,
   'average_neutral': 0.34935684005419415,
   'average_positive': 0.2741838240375121,
   'predicted_sentiment': 'negative'}},
 {'ID': 1,
  'Sentiment': {'average_negative': 0.7347313165664673,
   'average_neutral': 0.20912909507751465,
   'average_positive': 0.22280615599205097,
   'predicted_sentiment': 'negative'}},
 {'ID': 2,
  'Sentiment': {'average_negative': 0.41618318234880763,
   'average_neutral': 0.38278794785340625,
   'average_positive': 0.36769557061294716,
   'predicted_sentiment': 'negative'}},
 {'ID': 3,
  'Sentiment': {'average_negative': 0.8094285850723585,
   'average_neutral': 0.24255354205767313,
   'average_positive': 0.11468449902410309,
   'predicted_sentiment': 'negative'}},
 {'ID': 4,
  'Sentiment': {'average_negative': 0.21157451843221983,
   'average_neutral': 0.22378676508863768,
   'average_positive': 0.7313053968052069,
   'predicted_sentiment': 'positive'}},
 {'ID': 5,
  'Sentiment': {'av

In [104]:
# Convert Customer only Lists and full transcript list to dataframes with relevant columns

df1 = pd.DataFrame([{'ID': item['ID'], 
                     'negative_FullText': item['Sentiment']['average_negative'],
                     'neutral_FullText': item['Sentiment']['average_neutral'], 
                     'positive_FullText': item['Sentiment']['average_positive']} for item in full_text_sentiments])

df2 = pd.DataFrame([{'ID': item['ID'], 
                     'negative_CustomerText': item['Sentiment']['average_negative'],
                     'neutral_CustomerText': item['Sentiment']['average_neutral'], 
                     'positive_CustomerText': item['Sentiment']['average_positive']} for item in customer_sentiments])

# Merge the two dataframes on 'ID'
merged_df_3splits = pd.merge(df1, df2, on='ID')

# Add the negative, neutral, and positive columns
merged_df_3splits['summed_negative'] = merged_df_3splits['negative_FullText'] + merged_df_3splits['negative_CustomerText']
merged_df_3splits['summed_neutral'] = merged_df_3splits['neutral_FullText'] + merged_df_3splits['neutral_CustomerText']
merged_df_3splits['summed_positive'] = merged_df_3splits['positive_FullText'] + merged_df_3splits['positive_CustomerText']

# Define function to assign sentiment based on max value

def get_sentiment(negative, neutral, positive):
    if positive > max(negative, neutral):
        return 'positive'
    elif negative > max(neutral, positive):
        return 'negative'
    else:
        return 'neutral'

# Apply the function for list1
merged_df_3splits['sentiment_FullText'] = merged_df.apply(
    lambda row: get_sentiment(row['negative_FullText'], row['neutral_FullText'], row['positive_FullText']), axis=1)

# Apply the function for list2
merged_df_3splits['sentiment_CustomerText'] = merged_df.apply(
    lambda row: get_sentiment(row['negative_CustomerText'], row['neutral_CustomerText'], row['positive_CustomerText']), axis=1)

# Apply the function for the summed values
merged_df_3splits['summed_sentiment'] = merged_df.apply(
    lambda row: get_sentiment(row['summed_negative'], row['summed_neutral'], row['summed_positive']), axis=1)

# Save the dataframe to a CSV file

merged_df_3splits.head(20)


,ID,negative_FullText,neutral_FullText,positive_FullText,negative_CustomerText,neutral_CustomerText,positive_CustomerText,summed_negative,summed_neutral,summed_positive,sentiment_FullText,sentiment_CustomerText,summed_sentiment
0,0,0.543126,0.349357,0.274184,0.301774,0.245986,0.618906,0.844900,0.595343,0.893090,negative,positive,positive
1,1,0.734731,0.209129,0.222806,0.611135,0.165312,0.390220,1.345866,0.374441,0.613026,negative,negative,negative
2,2,0.416183,0.382788,0.367696,0.321756,0.387430,0.457480,0.737939,0.770218,0.825175,negative,positive,negative
3,3,0.809429,0.242554,0.114684,0.961567,0.114221,0.090879,1.770996,0.356774,0.205563,negative,negative,negative
4,4,0.211575,0.223787,0.731305,0.313439,0.389511,0.463717,0.525014,0.613298,1.195022,positive,positive,positive
5,5,0.246542,0.368938,0.551186,0.211929,0.366109,0.588629,0.458471,0.735047,1.139815,positive,positive,positive
6,6,0.447539,0.323811,0.395317,0.585790,0.374040,0.206836,1.033329,0.697851,0.602153,positive,negative,negative
7,7,0.332144,0.412612,0.421911,0.286550,0.598944,0.281173,0.618694,1.011555,0.703084,positive,positive,positive
8,8,0.394793,0.494662,0.277212,0.250743,0.603372,0.312552,0.645536,1.098033,0.589764,positive,positive,positive
9,9,0.426608,0.334073,0.405986,0.624246,0.375301,0.167120,1.050854,0.709374,0.573106,negative,neutral,negative


In [105]:
merged_df_3splits

,ID,negative_FullText,neutral_FullText,positive_FullText,negative_CustomerText,neutral_CustomerText,positive_CustomerText,summed_negative,summed_neutral,summed_positive,sentiment_FullText,sentiment_CustomerText,summed_sentiment
0,0,0.543126,0.349357,0.274184,0.301774,0.245986,0.618906,0.844900,0.595343,0.893090,negative,positive,positive
1,1,0.734731,0.209129,0.222806,0.611135,0.165312,0.390220,1.345866,0.374441,0.613026,negative,negative,negative
2,2,0.416183,0.382788,0.367696,0.321756,0.387430,0.457480,0.737939,0.770218,0.825175,negative,positive,negative
3,3,0.809429,0.242554,0.114684,0.961567,0.114221,0.090879,1.770996,0.356774,0.205563,negative,negative,negative
4,4,0.211575,0.223787,0.731305,0.313439,0.389511,0.463717,0.525014,0.613298,1.195022,positive,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.566405,0.251040,0.349222,0.620798,0.340665,0.205204,1.187203,0.591705,0.554426,negative,negative,negative
196,196,0.391895,0.485025,0.289746,0.442110,0.425744,0.298813,0.834005,0.910769,0.588559,positive,positive,positive
197,197,0.336472,0.357911,0.472283,0.399684,0.475559,0.291423,0.736156,0.833470,0.763707,positive,positive,positive
198,198,0.518528,0.187770,0.460369,0.423803,0.169055,0.573809,0.942331,0.356825,1.034177,negative,positive,positive


In [106]:
# Save the dataframe to a CSV file
merged_df_3splits.to_csv('sentiment_data_3splits_weighted.csv', index=False)

### Without splitting each documents into 3 sections

In [10]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Function to compute sentiment with custom label grouping
def custom_sentiment_analysis(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Get the logits from the model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Convert logits to probabilities using softmax
    probs = F.softmax(logits, dim=-1).squeeze().tolist()

    # Combine label 1 and 2 as Negative, label 3 as Neutral, label 4 and 5 as Positive
    negative_prob = probs[0] + probs[1]  # Combine 1-star and 2-star for Negative
    neutral_prob = probs[2]               # 3-star is Neutral
    positive_prob = probs[3] + probs[4]   # Combine 4-star and 5-star for Positive

    # Return the combined sentiment probabilities
    return {
        "negative": negative_prob,
        "neutral": neutral_prob,
        "positive": positive_prob,
        "all_probs": {
            "1_star": probs[0],
            "2_star": probs[1],
            "3_star": probs[2],
            "4_star": probs[3],
            "5_star": probs[4]
        }
    }


### Customers Only transcripts

In [54]:
import os

# Directory where customer-only transcripts are stored
customer_text_dir = './customer_only_transcripts/'

# Load all customer texts from the directory
customer_texts = []
for i in range(200):  # Assuming 200 files
    file_path = os.path.join(customer_text_dir, f'transcript_{i}_customer.txt')
    with open(file_path, 'r', encoding='utf-8') as file:
        customer_texts.append(file.read())

In [55]:
len(customer_texts)

200

In [56]:
customer_texts[1]

"Hi, I'm calling about a denied claim I received for my recent medical service. I was told that my policy doesn't cover it, but I'm certain it should be covered under my new policy. My member ID is MEM123456.\nThe claim number is CLM789012, and the date of service was February 10th.\nI switched policies on January 1st. I was told that the new policy would cover the service I received, but the denial letter says it's not covered.\nThat's frustrating. How can we get this resolved?\nNo, that's all. Thank you for your help.\nThank you."

In [58]:
customer_sentiments = []

for idx, text in enumerate(customer_texts, start=0):  # Start index from 0
    sentiment = custom_sentiment_analysis(text)
    sentiment_entry = {"ID": idx, "Sentiment": sentiment}
    customer_sentiments.append(sentiment_entry)


# Print the results
for entry in customer_sentiments:
    print(f"ID: {entry['ID']}, Sentiment: {entry['Sentiment']}")


ID: 1, Sentiment: {'negative': 0.2108418345451355, 'neutral': 0.26327478885650635, 'positive': 0.5258834064006805, 'all_probs': {'1_star': 0.08516666293144226, '2_star': 0.12567517161369324, '3_star': 0.26327478885650635, '4_star': 0.3509575426578522, '5_star': 0.17492586374282837}}
ID: 2, Sentiment: {'negative': 0.7648677676916122, 'neutral': 0.12402941286563873, 'positive': 0.11110279709100723, 'all_probs': {'1_star': 0.5470452308654785, '2_star': 0.21782253682613373, '3_star': 0.12402941286563873, '4_star': 0.06711259484291077, '5_star': 0.043990202248096466}}
ID: 3, Sentiment: {'negative': 0.3537796437740326, 'neutral': 0.2893284261226654, 'positive': 0.356891930103302, 'all_probs': {'1_star': 0.15992584824562073, '2_star': 0.19385379552841187, '3_star': 0.2893284261226654, '4_star': 0.23878873884677887, '5_star': 0.11810319125652313}}
ID: 4, Sentiment: {'negative': 0.8932154029607773, 'neutral': 0.07842424511909485, 'positive': 0.028360378928482533, 'all_probs': {'1_star': 0.65918

### Full transcripts

In [61]:

full_text_dir = './transcripts_v3/'  # Directory where full transcripts are stored

# Load all customer texts from the directory
full_texts = []

for i in range(200):  # Assuming 200 files
    file_path = os.path.join(full_text_dir, f'transcript_{i}.txt')
    with open(file_path, 'r', encoding='utf-8') as file:
        full_texts.append(file.read())

In [62]:
full_text_sentiments = []

for idx, text in enumerate(full_texts, start=1):  # Start index from 1
    sentiment = custom_sentiment_analysis(text)
    sentiment_entry = {"ID": idx, "Sentiment": sentiment}
    full_text_sentiments.append(sentiment_entry)


# Print the results
for entry in full_text_sentiments:
    print(f"ID: {entry['ID']}, Sentiment: {entry['Sentiment']}")

ID: 1, Sentiment: {'negative': 0.3891886919736862, 'neutral': 0.23770961165428162, 'positive': 0.37310171127319336, 'all_probs': {'1_star': 0.1851586550474167, '2_star': 0.20403003692626953, '3_star': 0.23770961165428162, '4_star': 0.2542724907398224, '5_star': 0.11882922053337097}}
ID: 2, Sentiment: {'negative': 0.5301426351070404, 'neutral': 0.151644766330719, 'positive': 0.3182126134634018, 'all_probs': {'1_star': 0.3409744203090668, '2_star': 0.18916821479797363, '3_star': 0.151644766330719, '4_star': 0.17844991385936737, '5_star': 0.13976269960403442}}
ID: 3, Sentiment: {'negative': 0.4393303692340851, 'neutral': 0.240931436419487, 'positive': 0.3197382539510727, 'all_probs': {'1_star': 0.218072772026062, '2_star': 0.22125759720802307, '3_star': 0.240931436419487, '4_star': 0.217904195189476, '5_star': 0.10183405876159668}}
ID: 4, Sentiment: {'negative': 0.7671070396900177, 'neutral': 0.15668506920337677, 'positive': 0.07620786316692829, 'all_probs': {'1_star': 0.47235971689224243

In [66]:
# Convert Customer only Lists and full transcript list to dataframes with relevant columns

df1 = pd.DataFrame([{'ID': item['ID'], 
                     'negative_FullText': item['Sentiment']['negative'],
                     'neutral_FullText': item['Sentiment']['neutral'], 
                     'positive_FullText': item['Sentiment']['positive']} for item in full_text_sentiments])

df2 = pd.DataFrame([{'ID': item['ID'], 
                     'negative_CustomerText': item['Sentiment']['negative'],
                     'neutral_CustomerText': item['Sentiment']['neutral'], 
                     'positive_CustomerText': item['Sentiment']['positive']} for item in customer_sentiments])

# Merge the two dataframes on 'ID'
merged_df = pd.merge(df1, df2, on='ID')

# Add the negative, neutral, and positive columns
merged_df['summed_negative'] = merged_df['negative_FullText'] + merged_df['negative_CustomerText']
merged_df['summed_neutral'] = merged_df['neutral_FullText'] + merged_df['neutral_CustomerText']
merged_df['summed_positive'] = merged_df['positive_FullText'] + merged_df['positive_CustomerText']

# Define function to assign sentiment based on max value

def get_sentiment(negative, neutral, positive):
    if positive > max(negative, neutral):
        return 'positive'
    elif negative > max(neutral, positive):
        return 'negative'
    else:
        return 'neutral'

# Apply the function for list1
merged_df['sentiment_FullText'] = merged_df.apply(
    lambda row: get_sentiment(row['negative_FullText'], row['neutral_FullText'], row['positive_FullText']), axis=1)

# Apply the function for list2
merged_df['sentiment_CustomerText'] = merged_df.apply(
    lambda row: get_sentiment(row['negative_CustomerText'], row['neutral_CustomerText'], row['positive_CustomerText']), axis=1)

# Apply the function for the summed values
merged_df['summed_sentiment'] = merged_df.apply(
    lambda row: get_sentiment(row['summed_negative'], row['summed_neutral'], row['summed_positive']), axis=1)


merged_df.head(20)


,ID,negative_FullText,neutral_FullText,positive_FullText,negative_CustomerText,neutral_CustomerText,positive_CustomerText,summed_negative,summed_neutral,summed_positive,sentiment_FullText,sentiment_CustomerText,summed_sentiment
0,1,0.389189,0.237710,0.373102,0.210842,0.263275,0.525883,0.600031,0.500984,0.898985,negative,positive,positive
1,2,0.530143,0.151645,0.318213,0.764868,0.124029,0.111103,1.295010,0.275674,0.429315,negative,negative,negative
2,3,0.439330,0.240931,0.319738,0.353780,0.289328,0.356892,0.793110,0.530260,0.676630,negative,positive,negative
3,4,0.767107,0.156685,0.076208,0.893215,0.078424,0.028360,1.660322,0.235109,0.104568,negative,negative,negative
4,5,0.210432,0.216309,0.573259,0.216275,0.195495,0.588230,0.426707,0.411804,1.161489,positive,positive,positive
5,6,0.331262,0.280050,0.388688,0.088175,0.308399,0.603426,0.419438,0.588448,0.992114,positive,positive,positive
6,7,0.384551,0.218411,0.397038,0.368021,0.290793,0.341186,0.752572,0.509204,0.738224,positive,negative,negative
7,8,0.273150,0.238623,0.488227,0.133797,0.398228,0.467975,0.406947,0.636851,0.956202,positive,positive,positive
8,9,0.246966,0.323970,0.429064,0.102614,0.340785,0.556601,0.349580,0.664755,0.985664,positive,positive,positive
9,10,0.387904,0.295101,0.316995,0.381728,0.448737,0.169535,0.769632,0.743838,0.486530,negative,neutral,negative


In [67]:
# Save the dataframe to a CSV file
merged_df.to_csv('sentiment_data.csv', index=False)

### OpenAI

In [26]:
import os
import openai
import pandas as pd
from tqdm import tqdm

# Directory where full transcripts are stored
full_text_dir = './transcripts_v3/'

from openai import OpenAI

client = OpenAI(api_key=os.environ["openai_axa_key"].replace('“', '').replace('”', '').strip())
                                
def analyze_text(text):
    prompt = f"""Analyze the following customer service transcript:{text}

Provide two pieces of information:
1. The sentiment of the member (positive, neutral, or negative)
2. The outcome (issue resolved or followup needed)

Format your response as:
Sentiment: [sentiment]
Outcome: [outcome]"""

    response = client.chat.completions.create(
        model= "gpt-4o-mini",#"gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": "You are a helpful assistant skilled in analyzing customer service transcripts. Don't hallucinate, if you don't know just say I Dont know."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [28]:
# List to store results
results = []

# Process each file
for i in tqdm(range(10)):  # Using tqdm for progress bar
    file_path = os.path.join(full_text_dir, f'transcript_{i}.txt')
    
    # Extract ID from filename
    file_id = i
    
    # Read the file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Analyze the text
    analysis = analyze_text(text)
    
    # Parse the analysis
    sentiment = analysis.split('\n')[0].split(': ')[1]
    outcome = analysis.split('\n')[1].split(': ')[1]
    
    # Append results
    results.append({
        'ID': file_id,
        'Sentiment': sentiment,
        'Outcome': outcome
    })

# Create DataFrame
df = pd.DataFrame(results)
df

100%|███████████████████████████████████████████| 10/10 [00:11<00:00,  1.11s/it]


,ID,Sentiment,Outcome
0,0,Positive,Followup needed
1,1,Negative,Followup needed
2,2,Neutral,Followup needed
3,3,Negative,Followup needed
4,4,Positive,Issue resolved
5,5,Positive,Followup needed
6,6,Positive,Issue resolved
7,7,Neutral,Followup needed
8,8,Neutral,Followup needed
9,9,Neutral,Followup needed


In [11]:
# Save to CSV
df.to_csv('sentiment_analysis_outcome_results.csv', index=False)